# What to Include in the Outcome Metrics for Depression Treatment?--Taking a Look at the 50% Improvements in Different Aspects of Depression


When discussing treatment outcomes, we tend to focus solely on symptom reduction. We typically assume that other aspects of improvements always follow core symptom reduction. However, researchers rarely examine the overlaps between core symptom reduction and different types of outcome improvements. 

Here, we examined whether other types of outcome improvements follow core symptom reduction (here, depressive symptoms) in patients with depression after completing a partial hospital program.  

We define clinically significant improvement as 50% of improvement in one measured domain. 

We examined the overlaps of individuals with  50% improvement in reduction of depressive symptoms with 50% of improvements in:
1. Reduction of non-depressive symptoms
2. Positive mental health
3. Functioning
4. Well-being

We also compared the global patient rating of overall improvement in each response group: 
1. Responders to both domains (i.e., reduction of depressive symptoms and another measured domain)
2. Responders to depressive symptom reduction only
3. Responders to the other domain only
4. Non-responders to both domains 



### Load Packages

In [107]:
import pandas as pd
from dfply import *
import numpy as np
from plotnine import *
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind

### Customize Notebook Displays

In [108]:
pd.options.display.max_seq_items=300
pd.options.mode.chained_assignment = None  # default='warn'
warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

### Load Datasets

In [118]:
#RDQ: Different Types of Treatment Outcomes
rdqPre = pd.read_spss('../Data/PrePostMeasures/RDQPre_1.sav') #Pre-treatment
rdqPost = pd.read_spss('../Data/PrePostMeasures/RDQPost_1.sav') #Post-treatment

#Discharge Package: Patient perceived satisfaction & Improvement 
satOld = pd.read_spss('../Data/PHPSatisfaction/EndofTreatmentSurveyData/EOT Verfication_1.sav')#Old data before 2020
satNew = pd.read_spss('../Data/DischargePacket/2022-06-06 dc.sav') #New data after 2020
#We swtiched the platform for data collection in 2020 so we have two separate datasets

#Diagnosis
dx = pd.read_spss('../Data/DemosDx/Diagnosis_1.sav') 

#Demographics
demo = pd.read_spss('../Data/DemosDx/Demographics Form_1.sav')

In [121]:
rdqPre.ID1

0          1.0
1          2.0
2          3.0
3          4.0
4          5.0
         ...  
7272    7273.0
7273    7274.0
7274    7275.0
7275    7276.0
7276    7277.0
Name: ID1, Length: 7277, dtype: float64

### Preprocess Data

In [110]:
#Change ID type from float to object
rdqPre.ID1 = rdqPre.ID1.astype('object') 
rdqPost.ID1 = rdqPost.ID1.astype('object') 

#Replace text with level orders 
rdqPre = rdqPre.replace(
    {'not at all or rarely true': 0, 'sometimes true': 1, 'often or almost always true': 2})
rdqPost = rdqPost.replace(
    {'not at all or rarely true': 0, 'sometimes true': 1, 'often or almost always true': 2})

#Convert reverse items in RDQ
for i in [29,30,48,49,50,51,52] :
    colname1 = 'rdqpre_' + str(i) + '_1'
    colname2 = 'rdqpost_' + str(i) + '_1'
    rdqPre[colname1] = abs(rdqPre[colname1].astype('float') - 2)
    rdqPost[colname2] = abs(rdqPost[colname2].astype('float') - 2)
    
#Rename variables to ensure consistency between the old and new datasets of the global rating of improvement
satGlobalOld = satOld.loc[:,['PHP_ID_1', 'IMPRV_1']].rename(columns = {'PHP_ID_1': 'ID1','IMPRV_1': 'imprv1'})
satGlobalNew = satNew.loc[:,['id1', 'imprv_1']].rename(columns = {'id1': 'ID1','imprv_1': 'imprv1'})

#Merge old and new discharge package (i.e., overall satisfaction/improvement) datasets
imprv = pd.concat([satGlobalOld, satGlobalNew],axis = 0) #Use concact to merge data by columns
imprv = imprv[imprv.ID1 != 0] #Remove invalid IDs
imprv.ID1 = imprv.ID1.astype('object') #Change ID type from float to object


Create the five domains of improvements:
1. Reduction in depressive symptoms
2. Reduction in non-depressive symptoms
3. Positive mental health
4. Functioning 
5. Well-being

In [111]:
rdqPre['pre_dsym'] = rdqPre.iloc[:,1:15].mean(axis = 1) #Pre-treatment Symptom Reduction in Depressive Symptoms
rdqPre['pre_ndsym'] = rdqPre.iloc[:,15:26].mean(axis = 1) #Pre-treatment Symptom Reduction in Non-Depressive Symptoms
rdqPre['pre_cope'] = rdqPre.iloc[:,26:31].mean(axis = 1) #Pre-treatment Coping
rdqPre['pre_pmh'] = rdqPre.iloc[:,31:43].mean(axis = 1) #Pre-treatment Positive Mental Health
rdqPre['pre_fun'] = rdqPre.iloc[:,43:53].mean(axis = 1) #Pre-treatment Functioning
rdqPre['pre_well'] = rdqPre.iloc[:,53:61].mean(axis = 1) #Pre-treatment Well-being

rdqPost['post_dsym'] = rdqPost.iloc[:,1:15].mean(axis = 1) #Post-treatment Symptom Reduction in Depressive Symptoms
rdqPost['post_ndsym'] = rdqPost.iloc[:,15:26].mean(axis = 1) #Post-treatment Symptom Reduction in Non-Depressive Symptoms
rdqPost['post_cope'] = rdqPost.iloc[:,26:31].mean(axis = 1)#Post-treatment Coping
rdqPost['post_pmh'] = rdqPost.iloc[:,31:43].mean(axis = 1) #Post-treatment Positive Mental Health
rdqPost['post_fun'] = rdqPost.iloc[:,43:53].mean(axis = 1) #Post-treatment Functioning
rdqPost['post_well'] = rdqPost.iloc[:,53:61].mean(axis = 1)#Post-treatment Well-being

In [112]:
rdqPre.iloc[:,1:15].mean(axis = 0)

Series([], dtype: float64)

In [113]:
rdqPost.iloc[:,1:15].dropna(how = 'any')

,rdqpost_1_1,rdqpost_2_1,rdqpost_3_1,rdqpost_4_1,rdqpost_5_1,rdqpost_6_1,rdqpost_7_1,rdqpost_8_1,rdqpost_9_1,rdqpost_10_1,rdqpost_11_1,rdqpost_12_1,rdqpost_13_1,rdqpost_14_1
0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0
3,2,2,2,2,0,0,1,2,1,1.0,2,2,2,2
4,2,2,2,0,2,1,0,2,2,2.0,2,2,1,1
26,2,2,2,0,0,1,0,2,2,2.0,1,1,2,0
29,0,1,0,0,0,1,0,0,0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7239,2,2,2,1,1,2,0,1,2,2.0,1,1,2,1
7240,1,0,0,0,0,0,0,1,0,1.0,0,0,0,0
7244,0,0,0,0,0,0,0,0,0,0.0,0,1,0,0
7248,0,0,0,0,0,0,0,0,1,1.0,1,0,0,0


Here we selected two groups of samples. 

One group contains all patients with a diagnosis of major depressive disorder (MDD). 
In contrast, the other group comprises only patients with MDD as their <i> primary </i> diagnosis (i.e., they primarily came here for depression).

In [114]:
#Find patients with MDD
def findMDDAll(df: pd.DataFrame) -> pd.DataFrame:
    return df[df.mddsnp_1.str.contains('Curr|Prin') | df.mddrnp_1.str.contains('Curr|Prin')|
        df.mddsp_1.str.contains('Curr|Prin')| df.mddrp_1.str.contains('Curr|Prin')]

#Find patients with MDD as their primary diagnosis
def findMDDCurrPrin(df: pd.DataFrame) -> pd.DataFrame:
    return df[df.mddsnp_1.str.contains('Curr, Prin') | df.mddrnp_1.str.contains('Curr, Prin')|
        df.mddsp_1.str.contains('Curr, Prin')| df.mddrp_1.str.contains('Curr, Prin')]
    

### Compute Sample Sizes

Compute the sample sizes of MDD patients and primary MDD patients who completed:
1. Pre-treatment assessments of different outcome domains
2. Post-treatment assessment of different outcome domains
3. Global patient ratings of overall improvements 
4. All the above three measurements 

In [115]:
#Complete Data of Post-treatment RDQ
rdqPreComp = rdqPre.filter(regex = 'ID1|rdqpre_|pre_')
rdqPreComp = rdqPreComp[rdqPreComp.isnull().sum(axis = 1)< 66]
rdqPreComp #5738


rdqPostComp = rdqPost.filter(regex = 'ID1|rdqpost_|post_')
rdqPostComp = rdqPostComp[rdqPostComp.isnull().sum(axis = 1)< 66]
rdqPostComp #4023


sample = pd.merge(rdqPreComp, rdqPostComp, on = 'ID1', how = 'inner')
sample = pd.merge(sample, dx, on = 'ID1', how = 'inner')
sample = pd.merge(sample, imprv, on = 'ID1', how = 'inner')
sample #2746

sample = findMDDCurrPrin(sample) #1269
sample = sample.filter(regex = 'ID1|rdqpre_|pre_|rdqpost_|post_')
sample.dropna() #847


,ID1,rdqpre_1_1,rdqpre_2_1,rdqpre_3_1,rdqpre_4_1,rdqpre_5_1,rdqpre_6_1,rdqpre_7_1,rdqpre_8_1,rdqpre_9_1,...,rdqpost_57_1,rdqpost_58_1,rdqpost_59_1,rdqpost_60_1,post_dsym,post_ndsym,post_cope,post_pmh,post_fun,post_well
4,1359.0,1,2,2,0,2,2,0,1,2,...,0,1,2,2,1.071429,0.727273,1.8,1.250000,1.1,1.000
9,1384.0,2,1,2,2,0,1,2,2,1,...,0,1,1,2,0.428571,0.818182,1.6,1.916667,0.9,1.250
12,1400.0,2,2,2,1,1,2,0,2,1,...,0,1,1,1,1.285714,1.636364,0.6,0.833333,0.6,0.875
16,1412.0,2,2,2,0,2,0,2,2,2,...,2,2,2,2,0.357143,0.727273,1.6,1.916667,2.0,2.000
20,1420.0,2,2,2,1,0,1,2,2,2,...,2,0,1,0,1.428571,1.000000,0.4,0.583333,1.1,0.375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2708,7173.0,1,1,2,0,0,0,1,2,1,...,0,1,1,1,0.714286,1.000000,0.6,0.916667,0.9,1.000
2711,7182.0,2,2,2,0,2,2,2,2,2,...,2,2,2,2,0.214286,0.272727,1.0,1.750000,2.0,2.000
2712,7185.0,2,2,2,2,1,2,2,2,1,...,0,0,2,0,0.857143,0.454545,1.8,0.666667,1.1,0.375
2717,7190.0,1,1,1,0,1,0,0,1,1,...,1,1,1,1,0.500000,0.181818,1.2,0.916667,1.3,1.125


In [104]:
847/1269

0.6674546887312844

In [87]:
#Complete Data of Pre-treatment RDQ
rdqPreComp = rdqPre.filter(regex = 'ID1|rdqpre_|pre_')
rdqPreComp = rdqPreComp.dropna() #Drop rows containing any missing values
rdqPreComp_dx = pd.merge(rdqPreComp, dx, on = 'ID1', how = 'inner') #Merge pre-treatment RDQ data with dianogses 

rdqPreCompMDDAll = findMDDAll(rdqPreComp_dx) #Find patients with MDD in the complete smaple
rdqPreCompMDDCurrPrin = findMDDCurrPrin(rdqPreComp_dx) #Find patients with primary MDD in the complete smaple


#Complete Data of Post-treatment RDQ
rdqPostComp = rdqPost.filter(regex = 'ID1|rdqpost|post_')
rdqPostComp = rdqPostComp.dropna() #Drop rows containing any missing values
rdqPostComp_dx = pd.merge(rdqPostComp, dx, on = 'ID1', how = 'inner') #Merge post-treatment RDQ data with dianogses 

rdqPostCompMDDAll = findMDDAll(rdqPostComp_dx) #Find patients with MDD in the complete smaple
rdqPostCompMDDCurrPrin = findMDDCurrPrin(rdqPostComp_dx) #Find patients with primary MDD in the complete smaple


#Complete Data of the Global Patient Rating of Improvement
imprvComp = imprv.dropna() #Drop rows containing any missing values
imprvComp_dx = pd.merge(imprvComp, dx, on = 'ID1', how = 'inner') #Merge post-treatment RDQ data with dianogses 

imprvCompMDDAll = findMDDAll(imprvComp_dx) #Find patients with MDD in the complete smaple
imprvCompMDDCurrPrin = findMDDCurrPrin(imprvComp_dx) #Find patients with primary MDD in the complete smaple

#Complete Data of the Above Assessments
rdqImprvDxComp = rdqPreComp.merge(rdqPostComp, on = 'ID1', how = 'inner') #Inner merging pre- and post-treatment RDQ
rdqImprvDxComp  = rdqImprvDxComp.merge(imprvComp, on = 'ID1', how = 'inner') #Inner merging Improvement
rdqImprvDxComp  = rdqImprvDxComp.merge(dx, on = 'ID1', how = 'inner') #Inner merging diagnoses

rdqImprvDxCompMDDAll = findMDDAll(rdqImprvDxComp)  #Find patients with MDD in the complete smaple
rdqImprvDxCompMDDCurrPrin = findMDDCurrPrin(rdqImprvDxComp) #Find patients with primary MDD in the complete smaple

sampleSize = pd.DataFrame([[rdqPreCompMDDAll.shape[0], rdqPreCompMDDCurrPrin.shape[0]],
                           [rdqPostCompMDDAll.shape[0], rdqPostCompMDDCurrPrin.shape[0]],
                           [imprvCompMDDAll.shape[0], imprvCompMDDCurrPrin.shape[0]],
                           [rdqImprvDxCompMDDAll.shape[0], rdqImprvDxCompMDDCurrPrin.shape[0]]],
                          columns = ['MDD ALL', 'PRINCIPLE MDD'],
                          index = ['PRE RDQ', 'POST RDQ', 'IMPROVEMENT', 'ALL ABOVE'])

sampleSize

3420


,MDD ALL,PRINCIPLE MDD
PRE RDQ,2626,1984
POST RDQ,1926,1441
IMPROVEMENT,2118,1558
ALL ABOVE,1126,844


NameError: name 'log' is not defined

### Label Clinically Significant Improvement: 50% Improvement

In [16]:
for sample in ['MDDAll', 'MDDCurrPrin']:
    dfMDD = locals().get('rdqImprvDxComp'+sample)

    labels = ['dsym', 'ndsym', 'cope', 'pmh', 'fun', 'well']

    #Compute Change Scores (Post - Pre RDQ)
    for label in labels:
        dfMDD['change_'+ label] = dfMDD.filter(regex='^post_'+label).squeeze() - dfMDD.filter(regex='^pre_'+label).squeeze()

    #Reverse symptom reduction scores (high scores = better improvement)
    dfMDD.change_dsym = dfMDD.change_dsym*(-1) 
    dfMDD.change_ndsym = dfMDD.change_ndsym*(-1)


    cut = 0.5 #Set a cutoff point
    for label in labels:
        dfMDD['change_'+label+'_cut'] = dfMDD['change_'+label] >= cut
        dfMDD['change_'+label+'_cut'] = dfMDD['change_'+label+'_cut'].replace({True: 1, False: 0}).astype('category')

    dfMDD.to_csv('../Data/dfMDD' + sample + '_satisfaction_rdq_dx_demo.csv') #Save processed data for later use
    
    


### Compute the Mean of the Patient Global Rating of Improvement in Each Responder Groups 

In [17]:
for sample in ['MDDAll', 'MDDCurrPrin']:
    dfMDD = locals().get('rdqImprvDxComp'+sample)

    #Create a contingency table
    contingencyTable = percentageTable = pd.crosstab(dfMDD['change_dsym_cut'], dfMDD['change_'+'ndsym'+'_cut'])

    for label in labels[2:]:
        contingencyTable = pd.concat([contingencyTable, pd.crosstab(dfMDD['change_dsym_cut'], 
                                                                    dfMDD['change_'+label+'_cut'])], axis = 1)
        percentageTable = pd.concat([percentageTable, pd.crosstab(dfMDD['change_dsym_cut'], 
                                                                    dfMDD['change_'+label+'_cut'],
                                                                  normalize='all')], axis = 1)
    #Rearrange contingency table
    rearrangeCont = {}
    for i in range(len(labels)-1):
        cont = [contingencyTable.iloc[1,2*i+1],
               contingencyTable.iloc[1,2*i],
               contingencyTable.iloc[0, 2*i+1],
               contingencyTable.iloc[0,2*i]]
        rearrangeCont[labels[i+1]] = cont

    
    rearrangeCont = pd.DataFrame(rearrangeCont)
    locals()['rearrangeCont'+sample] = rearrangeCont

    #Create a mean table
    meanTable = {}
    for label in labels[1:]:
        mean = [dfMDD[(dfMDD['change_dsym_cut'] == 1) & (dfMDD['change_'+label+'_cut'] == 1)].imprv1.mean(),
                dfMDD[(dfMDD['change_dsym_cut'] == 1) & (dfMDD['change_'+label+'_cut'] == 0)].imprv1.mean(),
                dfMDD[(dfMDD['change_dsym_cut'] == 0) & (dfMDD['change_'+label+'_cut'] == 1)].imprv1.mean(),
                dfMDD[(dfMDD['change_dsym_cut'] == 0) & (dfMDD['change_'+label+'_cut'] == 0)].imprv1.mean()]
        meanTable[label] = mean
    meanTable = pd.DataFrame(meanTable)
    meanTable

    #Create a mean improvement table with sample size in parentheses
    meanNTable = meanTable.copy()
    for i in range(meanNTable.shape[0]):
        for j in range(meanNTable.shape[1]):
            meanNTable.iloc[i,j] = str(round(meanTable.iloc[i,j],1)) + ' (' + str(rearrangeCont.iloc[i,j]) +')'

    meanNTable = meanNTable.rename(index = {0:'both', 1:'dep', 2:'other', 3:'none'})
    print(sample)
    display(meanNTable)
    meanNTable.to_csv('../Results/'+sample+'_meanImprvNTable.csv') 


MDDAll


,ndsym,cope,pmh,fun,well
both,3.2 (475),3.2 (438),3.3 (491),3.3 (416),3.3 (510)
dep,3.0 (159),3.0 (196),2.8 (143),2.9 (218),2.8 (124)
other,2.5 (102),2.7 (159),2.8 (142),2.8 (119),2.9 (176)
none,2.3 (390),2.1 (333),2.1 (350),2.2 (373),2.0 (316)


MDDCurrPrin


,ndsym,cope,pmh,fun,well
both,3.2 (366),3.2 (335),3.3 (375),3.3 (317),3.3 (393)
dep,2.9 (117),3.0 (148),2.7 (108),2.9 (166),2.7 (90)
other,2.4 (71),2.7 (108),2.8 (99),2.8 (89),2.9 (126)
none,2.2 (290),2.1 (253),2.1 (262),2.1 (272),2.0 (235)


### Compute the Percentage of Higher Global Patient Ratings of Improvement in Each Responder Group

In [10]:
for sample in ['MDDAll', 'MDDCurrPrin']:
    dfMDD = locals().get('rdqImprvDxComp'+sample)
    rearrangeCont = locals()['rearrangeCont'+sample]
    
    #Dichotimized Improvement scores (3,4 = higher improvement) 
    #Compute the number of people with higher improvement socres in each responder group
    
    imprvN34Table = {}
    percTable = {}
    for label in labels[1:]:
        N = [sum(dfMDD[(dfMDD['change_dsym_cut'] == 1) & (dfMDD['change_'+label+'_cut'] == 1)].imprv1>=3),
                sum(dfMDD[(dfMDD['change_dsym_cut'] == 1) & (dfMDD['change_'+label+'_cut'] == 0)].imprv1>=3),
                sum(dfMDD[(dfMDD['change_dsym_cut'] == 0) & (dfMDD['change_'+label+'_cut'] == 1)].imprv1>=3),
                sum(dfMDD[(dfMDD['change_dsym_cut'] == 0) & (dfMDD['change_'+label+'_cut'] == 0)].imprv1>=3)]

        imprvN34Table[label] = N

    imprvN34Table = pd.DataFrame(imprvN34Table)
    
    #Compute the percentage of people with higher improvement scores in each responder group
    #Number of people with improvement ratings of 3 or 4 in each group/Number of responders in each group
    
    imprvPercNTable = imprvN34Table.copy()

    for i in range(imprvPercNTable.shape[0]):
        for j in range(imprvPercNTable.shape[1]):
            imprvPercNTable.iloc[i,j] = str(round(imprvN34Table.iloc[i,j]/rearrangeCont.iloc[i,j]*100,1)) + '% (' + str(rearrangeCont.iloc[i,j]) + ')'
    
    imprvPercNTable  = imprvPercNTable.rename(index = {0:'both', 1:'dep', 2:'other', 3:'none'})
    imprvPercNTable .to_csv('../Results/'+sample+'_imprvPercNTable.csv') 
    
    print(sample)
    display(imprvPercNTable)    

MDDAll


,ndsym,cope,pmh,fun,well
both,86.3% (475),86.3% (438),89.4% (491),87.3% (416),87.5% (510)
dep,76.1% (159),78.1% (196),64.3% (143),77.1% (218),68.5% (124)
other,54.9% (102),67.3% (159),70.4% (142),68.1% (119),74.4% (176)
none,45.4% (390),37.8% (333),38.0% (350),40.8% (373),32.3% (316)


MDDCurrPrin


,ndsym,cope,pmh,fun,well
both,85.8% (366),86.0% (335),89.1% (375),87.1% (317),86.8% (393)
dep,76.1% (117),77.7% (148),63.9% (108),76.5% (166),68.9% (90)
other,50.7% (71),65.7% (108),69.7% (99),67.4% (89),76.2% (126)
none,44.8% (290),37.5% (253),37.0% (262),39.0% (272),29.8% (235)


### Pairwise Comparisons between Each Responder Group

We can either first do an ANOVA followed by post-hoc pairwise comparisons for those with significant ANOVA results or directly perform pairwise comparisons. Given the large sample size in the current case, I would predict all ANOVA results to be significant. I would usually skip ANOVA in this case. Here, I am only putting down ANOVA code for demonstration purposes. 

In [11]:
#ANOVA
for sample in ['MDDAll', 'MDDCurrPrin']:
    dfMDD = locals().get('rdqImprvDxComp'+sample)

    #Create responder group labels
    for label in labels[1:]:
        dfMDD['respGroup_'+label] = 0
        dfMDD['respGroup_'+label][(dfMDD['change_dsym_cut'] == 1) & (dfMDD['change_'+label+'_cut'] == 1)] = 'both'
        dfMDD['respGroup_'+label][(dfMDD['change_dsym_cut'] == 1) & (dfMDD['change_'+label+'_cut'] == 0)] = 'dep'
        dfMDD['respGroup_'+label][(dfMDD['change_dsym_cut'] == 0) & (dfMDD['change_'+label+'_cut'] == 1)] = 'other'
        dfMDD['respGroup_'+label][(dfMDD['change_dsym_cut'] == 0) & (dfMDD['change_'+label+'_cut'] == 0)] = 'none'

    anovaTable = pd.DataFrame()
    for label in labels[1:]:
        model = ols('imprv1 ~ C(respGroup_'+label+')', data=dfMDD).fit()
        anova = sm.stats.anova_lm(model, type = 2)
        anova = anova[['sum_sq','df','mean_sq','F','PR(>F)']] 
        anovaTable = pd.concat([anovaTable, anova])
    print(sample)
    display(anovaTable)

MDDAll


,sum_sq,df,mean_sq,F,PR(>F)
C(respGroup_ndsym),213.646289,3.0,71.215430,84.645732,2.260267e-49
Residual,943.978045,1122.0,0.841335,NaN,NaN
C(respGroup_cope),245.064480,3.0,81.688160,100.436278,1.369894e-57
Residual,912.559854,1122.0,0.813333,NaN,NaN
C(respGroup_pmh),280.279624,3.0,93.426541,119.479354,3.780155e-67
Residual,877.344710,1122.0,0.781947,NaN,NaN
C(respGroup_fun),251.285972,3.0,83.761991,103.693011,2.987838e-59
Residual,906.338362,1122.0,0.807788,NaN,NaN
C(respGroup_well),302.513540,3.0,100.837847,132.310415,2.187827e-73
Residual,855.110793,1122.0,0.762131,NaN,NaN


MDDCurrPrin


,sum_sq,df,mean_sq,F,PR(>F)
C(respGroup_ndsym),170.474255,3.0,56.824752,67.256812,5.535918e-39
Residual,709.709395,840.0,0.844892,NaN,NaN
C(respGroup_cope),192.973675,3.0,64.324558,78.626084,7.826022e-45
Residual,687.209974,840.0,0.818107,NaN,NaN
C(respGroup_pmh),223.584816,3.0,74.528272,95.345507,4.108362e-53
Residual,656.598833,840.0,0.781665,NaN,NaN
C(respGroup_fun),208.028897,3.0,69.342966,86.658751,7.399623e-49
Residual,672.154752,840.0,0.800184,NaN,NaN
C(respGroup_well),248.148565,3.0,82.716188,109.933135,4.801290e-60
Residual,632.035084,840.0,0.752423,NaN,NaN


As expected, we see significant group differences in global patient improvement ratings in each outcome domain.

Next, we perform pairwise comparisons based on t-tests. I compute the p-values of t-tests and cohen's d for effect size, which is independent of sample size. 

In [12]:
#Fucntion to compute Cohen's d for effect size
def cohend(d1, d2):
    n1, n2 = len(d1), len(d2)
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    u1, u2 = np.mean(d1), np.mean(d2)
    return (u1 - u2) / s

In [13]:
for sample in ['MDDAll', 'MDDCurrPrin']:
    dfMDD = locals().get('rdqImprvDxComp'+sample)

    #Create responder group labels
    for label in labels[1:]:
        dfMDD['respGroup_'+label] = 0
        dfMDD['respGroup_'+label][(dfMDD['change_dsym_cut'] == 1) & (dfMDD['change_'+label+'_cut'] == 1)] = 'both'
        dfMDD['respGroup_'+label][(dfMDD['change_dsym_cut'] == 1) & (dfMDD['change_'+label+'_cut'] == 0)] = 'dep'
        dfMDD['respGroup_'+label][(dfMDD['change_dsym_cut'] == 0) & (dfMDD['change_'+label+'_cut'] == 1)] = 'other'
        dfMDD['respGroup_'+label][(dfMDD['change_dsym_cut'] == 0) & (dfMDD['change_'+label+'_cut'] == 0)] = 'none'

    dfMDD['imprv_cut'] = pd.cut(dfMDD['imprv1'], [-98,2,98], labels = [0,1]) #Dichotimized improvement scores
    dfMDD['imprv_cut'] = dfMDD['imprv_cut'].astype('float') 

    respGroup = ['both','dep','other','none'] 
    sigTable = {}
    sigTableCut= {}

    for label in labels[1:]:
        sig = ['N']*6
        k = 0
        sig_cut = ['N']*6
        rowname = []
        for i in range(len(respGroup)):
            for j in range(i+1, len(respGroup)):   
                
                #t-test for the mean of improvement scores between responder groups
                res = ttest_ind(dfMDD.loc[dfMDD['respGroup_'+label] == respGroup[i], 'imprv1'],
                         dfMDD.loc[dfMDD['respGroup_'+label] == respGroup[j], 'imprv1']) 
                #cohen's d for the mean of improvement scores between responder groups
                d = cohend(dfMDD.loc[dfMDD['respGroup_'+label] == respGroup[i], 'imprv1'],
                         dfMDD.loc[dfMDD['respGroup_'+label] == respGroup[j], 'imprv1']) 

                #t-test for the dichotimized improvement scores between responder groups
                res_c = ttest_ind(dfMDD.loc[dfMDD['respGroup_'+label] == respGroup[i], 'imprv_cut'],
                         dfMDD.loc[dfMDD['respGroup_'+label] == respGroup[j], 'imprv_cut'])
                #cohen's d for the dichotimized improvement scores between responder groups
                d_c = cohend(dfMDD.loc[dfMDD['respGroup_'+label] == respGroup[i], 'imprv_cut'],
                         dfMDD.loc[dfMDD['respGroup_'+label] == respGroup[j], 'imprv_cut'])


                #Record p-values
                sig[k] = 'p=' +str(round(res[1],3)) + '; d='+str(round(d,2))
                sig_cut[k] = 'p=' +str(round(res_c[1],3)) + '; d='+str(round(d_c,2))

                k+=1

                rowname.append(respGroup[i]+ '-' +respGroup[j])

        sigTable[label] = sig
        sigTableCut[label] = sig_cut


    renameIndex = {}
    for i in range(len(rowname)):
            renameIndex[i] = rowname[i]

    sigTable = pd.DataFrame(sigTable)
    sigTable = sigTable.rename(index = renameIndex)
    sigTable.to_csv('../Results/'+sample+'meanImprvPairWiseComp.csv')

    sigTableCut = pd.DataFrame(sigTableCut).rename(index = renameIndex)
    sigTableCut.to_csv('../Results/'+sample+'percImprvPairWiseComp.csv')
    
    print(sample,': Continuous Improvement Scores')
    display(sigTable)
    print(sample,': Dichotimized Improvement Scores')
    display(sigTableCut)

MDDAll : Continuous Improvement Scores


,ndsym,cope,pmh,fun,well
both-dep,p=0.0; d=0.37,p=0.004; d=0.25,p=0.0; d=0.71,p=0.0; d=0.47,p=0.0; d=0.64
both-other,p=0.0; d=0.89,p=0.0; d=0.59,p=0.0; d=0.6,p=0.0; d=0.62,p=0.0; d=0.52
both-none,p=0.0; d=1.05,p=0.0; d=1.22,p=0.0; d=1.33,p=0.0; d=1.2,p=0.0; d=1.42
dep-other,p=0.0; d=0.47,p=0.002; d=0.33,p=0.518; d=-0.08,p=0.128; d=0.17,p=0.394; d=-0.1
dep-none,p=0.0; d=0.67,p=0.0; d=0.93,p=0.0; d=0.63,p=0.0; d=0.78,p=0.0; d=0.75
other-none,p=0.026; d=0.25,p=0.0; d=0.6,p=0.0; d=0.68,p=0.0; d=0.59,p=0.0; d=0.84


MDDAll : Dichotimized Improvement Scores


,ndsym,cope,pmh,fun,well
both-dep,p=0.002; d=0.28,p=0.009; d=0.22,p=0.0; d=0.71,p=0.001; d=0.28,p=0.0; d=0.52
both-other,p=0.0; d=0.84,p=0.0; d=0.5,p=0.0; d=0.55,p=0.0; d=0.52,p=0.0; d=0.36
both-none,p=0.0; d=0.97,p=0.0; d=1.18,p=0.0; d=1.31,p=0.0; d=1.12,p=0.0; d=1.42
dep-other,p=0.0; d=0.46,p=0.023; d=0.24,p=0.275; d=-0.13,p=0.073; d=0.21,p=0.265; d=-0.13
dep-none,p=0.0; d=0.64,p=0.0; d=0.87,p=0.0; d=0.54,p=0.0; d=0.78,p=0.0; d=0.78
other-none,p=0.087; d=0.19,p=0.0; d=0.61,p=0.0; d=0.68,p=0.0; d=0.56,p=0.0; d=0.92


MDDCurrPrin : Continuous Improvement Scores


,ndsym,cope,pmh,fun,well
both-dep,p=0.0; d=0.4,p=0.002; d=0.31,p=0.0; d=0.78,p=0.0; d=0.54,p=0.0; d=0.67
both-other,p=0.0; d=1.0,p=0.0; d=0.65,p=0.0; d=0.64,p=0.0; d=0.64,p=0.0; d=0.48
both-none,p=0.0; d=1.08,p=0.0; d=1.24,p=0.0; d=1.36,p=0.0; d=1.28,p=0.0; d=1.48
dep-other,p=0.001; d=0.53,p=0.01; d=0.33,p=0.498; d=-0.09,p=0.332; d=0.13,p=0.245; d=-0.16
dep-none,p=0.0; d=0.66,p=0.0; d=0.9,p=0.0; d=0.61,p=0.0; d=0.8,p=0.0; d=0.79
other-none,p=0.18; d=0.18,p=0.0; d=0.58,p=0.0; d=0.67,p=0.0; d=0.64,p=0.0; d=0.93


MDDCurrPrin : Dichotimized Improvement Scores


,ndsym,cope,pmh,fun,well
both-dep,p=0.014; d=0.26,p=0.024; d=0.22,p=0.0; d=0.7,p=0.003; d=0.29,p=0.0; d=0.49
both-other,p=0.0; d=0.93,p=0.0; d=0.53,p=0.0; d=0.56,p=0.0; d=0.53,p=0.005; d=0.29
both-none,p=0.0; d=0.97,p=0.0; d=1.17,p=0.0; d=1.33,p=0.0; d=1.16,p=0.0; d=1.47
dep-other,p=0.0; d=0.55,p=0.034; d=0.27,p=0.378; d=-0.12,p=0.119; d=0.21,p=0.235; d=-0.16
dep-none,p=0.0; d=0.65,p=0.0; d=0.87,p=0.0; d=0.56,p=0.0; d=0.81,p=0.0; d=0.85
other-none,p=0.375; d=0.12,p=0.0; d=0.58,p=0.0; d=0.68,p=0.0; d=0.59,p=0.0; d=1.04


Overall, you can see that patients with 50% improvement in depressive symptom reduction and other outcome domains reported significantly higher global ratings of improvement than patients with improvements in only one outcome domain. Patients with 50% improvement in either depressive symptom reduction or other outcome domains reported higher global improvement ratings than nonresponders. Lastly, there was no significant difference in global improvement ratings between patients who only responded to depressive symptom reduction or other treatment outcome domains. 

<b> Such findings indicate that the improvement in symptom reduction is no more critical than other treatment outcome domains. We should consider adding other treatment outcome domains to standard outcome metrics to capture the full spectrum of depression remission. </b>


In [14]:
dfMDD.filter(regex= 'change_.*_cut')

,change_dsym_cut,change_ndsym_cut,change_cope_cut,change_pmh_cut,change_fun_cut,change_well_cut
1,1,0,1,1,1,1
4,1,1,1,1,0,1
7,0,0,1,1,0,1
11,1,1,1,1,1,1
14,0,0,0,0,0,0
...,...,...,...,...,...,...
1805,0,0,0,1,0,0
1806,1,1,0,1,1,1
1807,1,1,1,0,0,0
1810,0,0,0,0,0,0


In [21]:
dfMDD = rdqImprvDxCompMDDAll
dfMDD['pre_dsym']
dfMDD['post_dsym']


1       1.071429
2       1.357143
4       0.428571
7       1.285714
11      0.357143
          ...   
1806    0.214286
1807    0.857143
1810    0.500000
1811    0.500000
1812    0.714286
Name: post_dsym, Length: 1126, dtype: float64

In [28]:
rdqPre.rdqpre_10_1

0       NaN
1       NaN
2         2
3         2
4         2
       ... 
7272      1
7273      2
7274      1
7275      2
7276      2
Name: rdqpre_10_1, Length: 7277, dtype: category
Categories (3, int64): [0, 2, 1]